In [ ]:
import torch
import argparse
import wandb
from typing import Any, List, Tuple

from dataclasses import dataclass, field
from typing import Dict, Tuple, Any, Optional
from torch.optim.optimizer import Optimizer, required


import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from nadir import tests
from nadir import nadir as optim
from nadir.tests import mnist

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
args = argparse.Namespace()
args.random_seeds : List[int] = [42]
args.seed : int = args.random_seeds[0]

args.learning_rate : float = 1e-3
args.batch_size : int = 64
args.test_batch_size : int = 1000
args.gamma : float = 0.7
args.device : bool = 'cuda' if torch.cuda.is_available() else 'cpu'
args.log_interval : int = 10
args.epochs : int = 10
args.betas : Tuple[float, float] = (0.9, 0.99)
args.eps : float = 1e-16
args.optimizer : Any = optim.Adam


In [ ]:
largs = argparse.Namespace()
largs.run_name : str = 'DoE-Adam'
largs.run_seed : str = args.seed

In [ ]:
run = wandb.init(project="MNIST", entity="dawn-of-eve")
run.name = f'{largs.run_name}'
run.config.update(args)
run.config.update(largs)

In [ ]:
mnist.mnist_tester(args.optimizer, args, largs)
